<a href="https://colab.research.google.com/github/Rishabh66/breast_Cancer/blob/main/ANNinPytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import sklearn.datasets as datasets
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [6]:
# will use Mnist dataset which have 70k images, but will train our module with 6000 Images

# Random seeds

torch.manual_seed(42)

In [7]:
df = pd.read_csv('fmnist_small.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [8]:
X = df.iloc[:,1:].values
y = df.iloc[:,0].values

In [9]:
X.shape

(6000, 784)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [11]:
# scaling the features
X_train = X_train/255
X_test = X_test/255

In [12]:
# Create a Customer Dataset
class CustomerDataset(Dataset):
  def __init__(self, data, lables):
    self.data = torch.tensor(data, dtype=torch.float32)
    self.labels = torch.tensor(lables, dtype=torch.long)

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    return self.data[index], self.labels[index]

In [13]:
# create a object for CustomerDataset class
traindataset = CustomerDataset(X_train, y_train)
testdataset = CustomerDataset(X_test, y_test)

In [14]:
len(traindataset)

4800

In [15]:
traindataset[0]

(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [16]:
# create dataloader Object
trainloader = DataLoader(traindataset, batch_size=64, shuffle = True)
testloader = DataLoader(testdataset, batch_size = 4, shuffle = False)


In [17]:
# create a NN Class and then a Training loop

class MyNN(nn.Module):
  def __init__(self, input_features=784):
    super().__init__()

    self.model = nn.Sequential(
        nn.Linear(input_features, 128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,10)
    )
  def forward(self,x):
        return self.model(x)

In [18]:
# set parameters
epochs = 100
learning_rate = 0.1

In [19]:
# initiate model creating object of model

model = MyNN(X_train.shape[1])

# loss function
criteria = nn.CrossEntropyLoss()

# optimizer
optimizer = optim.SGD(model.parameters(), lr=learning_rate)


In [20]:
# Creating Trainging Loops
for epoch in range(epochs):
  total_epoch_loss = 0

  for batch_features, batch_lables in trainloader:

    # Forward pass
    model_output = model(batch_features)

    # Calculate Loss
    loss = criteria(model_output, batch_lables)

    # zero gradients
    optimizer.zero_grad()

    # Back Pass
    loss.backward()

    # update grads
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  total_epoch_loss = total_epoch_loss/len(trainloader)
  print(f'Epoch {epoch+1} Loss {total_epoch_loss}')



Epoch 1 Loss 1.6442172074317931
Epoch 2 Loss 0.9228938269615173
Epoch 3 Loss 0.7810722343126932
Epoch 4 Loss 0.6865317821502686
Epoch 5 Loss 0.6385788853963216
Epoch 6 Loss 0.5826328301429748
Epoch 7 Loss 0.556782173315684
Epoch 8 Loss 0.5279252628485361
Epoch 9 Loss 0.5011282893021901
Epoch 10 Loss 0.4925721691052119
Epoch 11 Loss 0.46334261655807496
Epoch 12 Loss 0.4693782216310501
Epoch 13 Loss 0.42871190031369527
Epoch 14 Loss 0.4251930969953537
Epoch 15 Loss 0.39448946992556255
Epoch 16 Loss 0.4031505590677261
Epoch 17 Loss 0.3765400765339533
Epoch 18 Loss 0.3678211053212484
Epoch 19 Loss 0.35889905800422034
Epoch 20 Loss 0.3432086149851481
Epoch 21 Loss 0.34271817286809286
Epoch 22 Loss 0.32998157183329263
Epoch 23 Loss 0.31832168142000833
Epoch 24 Loss 0.31443997522195183
Epoch 25 Loss 0.30276435653368633
Epoch 26 Loss 0.2945211269458135
Epoch 27 Loss 0.2889769389232
Epoch 28 Loss 0.2907022300362587
Epoch 29 Loss 0.26991044004758197
Epoch 30 Loss 0.26457142372926074
Epoch 31 Los

In [21]:
# set model to eval mode
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [23]:
# Evalution Code will calculate accuracy

total = 0
correct_variable = 0

with torch.no_grad():
  for batch_features, batch_lables in testloader:
    model_output = model(batch_features)

    _, predicted = torch.max(model_output, 1)
    total = total + batch_lables.size(0)
    correct_variable = correct_variable + (torch.max(model_output, 1)[1] == batch_lables).sum().item()

print(correct_variable/total)


0.8275
